In [1]:
import pymc3 as pm 
import matplotlib.pyplot as plt
import numpy as np 
import theano
import theano.tensor as tt
from scipy import stats

In [2]:
def counterfactual_sample(data,trace,num_extra_unobserved=20,number_of_chains=2):
    
    
    
    num_original_sample = data['view'].shape[0]
    print('num_original_sample is ', num_original_sample)
    
    mu_mult = np.zeros(num_original_sample)
    cov_mult = np.eye(num_original_sample)
    
    transcript0 = trace["transcript0"].mean(axis=0)
    view0 = trace["view0"].mean(axis=0)
    eta_u_transcript = trace["eta_u_transcript"].mean(axis=0)
    eta_u_view = trace["eta_u_view"].mean(axis=0)
    eta_u_rating =trace["eta_u_rating"].mean(axis=0)
    eta_a_transcript = trace["eta_a_transcript"].mean(axis=0)
    eta_a_view = trace["eta_a_view"].mean(axis=0)
    eta_a_rating = trace["eta_a_rating"].mean(axis=0)
    eta_transcript_view = trace["eta_transcript_view"].mean(axis=0)
    eta_transcript_rating = trace["eta_transcript_rating"].mean(axis=0)
    eta_view_rating = trace["eta_view_rating"].mean(axis=0)
    sigma_transcript_sq = trace["sigma_transcript_sq"].mean(axis=0)
    sigma_rating_sq = trace["sigma_rating_sq"].mean(axis=0)
    u_list = []
    
    #ABDUCTION part
    
    with pm.Model() as model:

        #prior sampling
        u = pm.MvNormal('u',mu=mu_mult,cov=cov_mult,shape=num_original_sample)

        transcript_mean = transcript0 + eta_u_transcript * u + tt.dot(data['a'] , eta_a_transcript)
        transcript = pm.MvNormal('transcript', mu= transcript_mean, cov = sigma_transcript_sq*cov_mult, observed = data["transcript"] )

        view_mean = tt.exp(view0 + eta_u_view * u +  transcript * eta_transcript_view + tt.dot(data['a'],eta_a_view))
        view = pm.Poisson('view',mu =view_mean, observed = data['view'] )

        rating_mean = eta_u_rating * u +  tt.dot(data['a'],eta_a_rating) + transcript* eta_transcript_rating + view* eta_view_rating 
        rating = pm.MvNormal('rating', mu= rating_mean, cov = sigma_rating_sq*cov_mult, observed = data["rating"] ) 

        new_trace = pm.sample(num_extra_unobserved,chains=number_of_chains)
        u_list=new_trace['u']
        
    
#     temp_u = np.zeros((num_extra_unobserved,num_original_sample))
#     for i in range(number_of_chains):
#         temp_u =temp_u + u_list[i*num_extra_unobserved:(i+1)*num_extra_unobserved]
        
#     u_list =(1.0/number_of_chains)* temp_u.T
        

        
#     #ACTION part
#     new_data["u"] = np.array([])
#     new_data["a"] = np.array([])
#     new_data["transcript"] = np.array([])
#     new_data["view"] = np.array([])
    
#     num_repeat = 12*num_extra_unobserved
    
#     for i in range(num_original_sample):
        
#         new_data["transcript"] = new_data["transcript"]+num_repeat*[data['transcript'][i]]
#         new_data["view"] = new_data["view"]+num_repeat*[data['view'][i]]
        
#         new_data["u"] = new_data["u"] + 12*[u_list[i]]
#         new_data["a"] = 
    
#     #PREDICTION part
#     new_data["rating"] = np.array([])
            
    return u_list

In [ ]:
#     for i in range(num_original_sample):
        
#         with pm.Model() as model:

#             #prior sampling
#             u = pm.Normal('u',mu=0,tau=1)

#             transcript_mean = transcript0 + eta_u_transcript * u + tt.dot(data['a'][i] , eta_a_transcript)
#             transcript = pm.Normal('transcript', mu= transcript_mean, sd = sigma_transcript_sq, observed = data["transcript"][i] )

#             view_mean = tt.exp(view0 + eta_u_view * u +  transcript * eta_transcript_view + tt.dot(data['a'][i],eta_a_view))
#             view = pm.Poisson('view',mu =view_mean, observed = data['view'][i] )

#             rating_mean = eta_u_rating * u +  tt.dot(data['a'][i],eta_a_rating) + transcript* eta_transcript_rating + view* eta_view_rating 
#             rating = pm.Normal('rating', mu= rating_mean, sd = sigma_rating_sq, observed = data["rating"][i] ) 

#             new_trace = pm.sample(num_extra_unobserved)
#             print('sampling for ',i,' th time')
#             u_list.append(new_trace['u'])